# Задание 1 (10 баллов)

Обучите языковую модель с помощью GRPO на вот этом датасете - `notbadai/python_functions_reasoning`
Он чем-то похож на датасет из семинара, но вместо математических задач нужно написать код на питоне. 
Измените system prompt под эту задачу. Напишите аналогичные reward функции - одна на проверку формата (markdown wrappers ```python... ``` вокруг кода в конце), и на проверку что код работает (можете использовать import ast; ast.parse(string) для проверки)
Если модель сначала не может отвечать в нужно формате, то сделайте цикл SFT обучения на небольшом куске датасета.
GRPO в колабе может работать долго, поэтому не старайтесь пропустить весь датасет. Но обучайте хотя бы на 100 промптах (по несколько генераций на каждый промпт).
Оцените разницу в качестве на небольшом сабсете, прогнав изначальную модель, модель после SFT (если она есть) и GRPO модель. В качестве метрики используйте reward функции. 
Можете взять любую языковую модель любого размера. Можете дообучать всю модель или же только адаптера поверх квантизированной модели. 

In [ ]:
import re
import ast
import pycodestyle

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import GRPOConfig, GRPOTrainer
from trl import SFTConfig, SFTTrainer

from datasets import load_dataset
from peft import LoraConfig

from tqdm import tqdm
from collections import Counter

In [6]:
model_name = "Qwen/Qwen2-0.5B-Instruct"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

device

'cuda'

In [39]:
class Rewarder:

    # На самом деле это проверит еще и наличие ризонинга,
    # но будто бы так даже лучше
    pattern = re.compile(
        r"(?P<reasoning>.*)```python(?P<code>.*?)```</s>",
        re.MULTILINE | re.DOTALL
    )

    @classmethod
    def format_and_answer_accuracy_func(cls, completions, **kwargs):
        rewards = []

        for response in completions:
            m = cls.pattern.match(response)

            # Проверка на формат
            if m is None:
                rewards.append(0.0)

            else:
                rewards.append(1.0)

                try:
                    #проверка на работоспособность 
                    ast.parse(m["code"])
                    rewards[-1] += 2.0

                    # проверка pep8
                    style_checker = pycodestyle.Checker(lines=[m["code"]])
                    rewards[-1] += (0.5, 0.0)[style_checker.check_all(expected="E501") > 0]
                
                except Exception:
                    rewards[-1] += 0.0
           
        return rewards

In [8]:
dataset = load_dataset('notbadai/python_functions_reasoning')

README.md:   0%|          | 0.00/668 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/414M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/206299 [00:00<?, ? examples/s]

In [9]:
dataset['train'][0]

{'prompt': '```python\ndef scientific_to_float(scientific_notation):\n    """Create a script to convert a number in scientific notation into a float, considering the following conditions:\n- The script should handle both positive and negative exponents.\n- The script should handle scientific notation with multiple digits before the "e" symbol.\n- The script should handle scientific notation with a decimal point before the "e" symbol.\n- The script should handle scientific notation with multiple digits after the decimal point.\n- The script should handle scientific notation with a negative exponent that requires more than one digit.\n- The script should handle scientific notation with a decimal point after the "e" symbol.\n- The script should handle scientific notation with multiple digits after the decimal point and a negative exponent.\n- The script should handle scientific notation with multiple digits after the decimal point and a positive exponent.\n- The script should handle scien

In [10]:
system_prompt = """
You task is to write python code. Before outputting the result, give step-by-step reasoning.
It is wery important to do the job right. 10 kittens will die each time you fail.
Put the final asnwer at the end in markdown wrappers like this:
```python
*answer*
```
"""

def chatml_format_sft(example, system_prompt=system_prompt):
    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": example['prompt']
        },
        {
            "role": "assistant",
            "content": f"{example['reasoning']}\n{example['answer']}"
        },
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return {
        "text": prompt,
    }

def chatml_format_grpo(example, system_prompt=system_prompt):
    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": example['prompt']
        },
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return {
        "prompt": prompt,
    }

In [11]:
dataset_train = dataset['train'].select([i for i in range(100)])
dataset_test = dataset['train'].select([i for i in range(100, 200)])
dataset_train_for_sft = dataset['train'].select([i for i in range(200, 1000)])

original_columns = dataset_train_for_sft.column_names

# отформатируем данные в промпты
dataset_train = dataset_train.map(
    chatml_format_grpo,
    remove_columns=original_columns
)
dataset_test = dataset_test.map(
    chatml_format_grpo,
    remove_columns=original_columns
)
dataset_train_for_sft = dataset_train_for_sft.map(
    chatml_format_sft,
    remove_columns=original_columns
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
).to(device)

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [13]:
tot_rewards = []

for batch in tqdm(dataset_test):
    input_ids = tokenizer.batch_encode_plus([batch['prompt']], return_tensors='pt', padding=True)

    output = model.generate(
        input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
        max_new_tokens=1024, do_sample=True, temperature=1.5, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
    )

    completions = tokenizer.batch_decode(output, skip_special_tokens=False)

    tot_rewards.extend(Rewarder.format_and_answer_accuracy_func(completions))

print(sum(tot_rewards), Counter(tot_rewards))

100%|██████████| 100/100 [18:34<00:00, 11.14s/it]


NameError: name 'Counter' is not defined

In [15]:
print(sum(tot_rewards), Counter(tot_rewards))

0.0 Counter({0.0: 100})


In [16]:
training_args = SFTConfig(
    max_length=1024,
    label_names=["labels"],
    report_to="none",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10,
)

In [17]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=dataset_train_for_sft,
    args=training_args,
)

Converting train dataset to ChatML:   0%|          | 0/800 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

In [18]:
trainer.train()

Step,Training Loss
10,0.867900
20,0.635900
30,0.621700
40,0.632700
50,0.574900
60,0.628900
70,0.644500
80,0.607200
90,0.623800
100,0.627800


TrainOutput(global_step=600, training_loss=0.3607729834318161, metrics={'train_runtime': 852.6541, 'train_samples_per_second': 2.815, 'train_steps_per_second': 0.704, 'total_flos': 2689623638178048.0, 'train_loss': 0.3607729834318161})

In [44]:
tot_rewards = []

for batch in tqdm(dataset_test):
    input_ids = tokenizer.batch_encode_plus([batch['prompt']], return_tensors='pt', padding=True)

    output = model.generate(
        input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
        max_new_tokens=1024, do_sample=True, temperature=1.5, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
    )

    completions = tokenizer.batch_decode(output, skip_special_tokens=False)

    tot_rewards.extend(Rewarder.format_and_answer_accuracy_func(completions))

print(sum(tot_rewards), Counter(tot_rewards))

100%|██████████| 100/100 [23:21<00:00, 14.02s/it]

207.5 Counter({1.0: 50, 3.5: 45, 0.0: 5})


В 50 случаях модель верно составила ответ (ризонинг, затем код), еще в 45 она верно написала код и попала в pep8. В 5 случаях она не справилась вообще.

In [20]:
trainer.save_model("./Qwen-sft")

In [40]:
model_name = "./Qwen-sft"
device = 'cuda'

In [41]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [42]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
).to("cuda")

for param in model.parameters():
    param.requires_grad = False

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [45]:
dataset = load_dataset('notbadai/python_functions_reasoning')

dataset_train = dataset['train'].select([i for i in range(100)])
dataset_test = dataset['train'].select([i for i in range(100, 200)])
dataset_train_for_sft = dataset['train'].select([i for i in range(200, 1000)])

original_columns = dataset_train_for_sft.column_names

# отформатируем данные в промпты
dataset_train = dataset_train.map(
    chatml_format_grpo,
    remove_columns=original_columns
)
dataset_test = dataset_test.map(
    chatml_format_grpo,
    remove_columns=original_columns
)
dataset_train_for_sft = dataset_train_for_sft.map(
    chatml_format_sft,
    remove_columns=original_columns
)

In [46]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    ## https://huggingface.co/doubleyyh/email-tuned-qwen2-lora
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
    target_modules="all-linear"
)

In [47]:
training_args = GRPOConfig(
    output_dir="./Qwen-grpo",
    logging_steps=2,
    report_to="none",
    num_generations=6,
    num_train_epochs=1,
    temperature=1.5,
    per_device_train_batch_size=6,
    label_names=["labels"]
)

In [48]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[Rewarder.format_and_answer_accuracy_func],
    args=training_args,
    train_dataset=dataset_train,
    peft_config=peft_config
)

In [49]:
trainer.train()

Step,Training Loss
2,0.000000
4,0.000000
6,0.048800
8,0.030100
10,-0.000000
12,-0.000000
14,-0.000000
16,-0.000000
18,0.047700
20,-0.000000


TrainOutput(global_step=100, training_loss=0.00788883390814135, metrics={'train_runtime': 2476.7466, 'train_samples_per_second': 0.04, 'train_steps_per_second': 0.04, 'total_flos': 0.0, 'train_loss': 0.00788883390814135})

In [50]:
trainer.save_model("./Qwen-grpo")

In [51]:
tot_rewards = []

for batch in tqdm(dataset_test):
    input_ids = tokenizer.batch_encode_plus([batch['prompt']], return_tensors='pt', padding=True)

    output = model.generate(
        input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
        max_new_tokens=1024, do_sample=True, temperature=1.5, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
    )

    completions = tokenizer.batch_decode(output, skip_special_tokens=False)

    tot_rewards.extend(Rewarder.format_and_answer_accuracy_func(completions))

print(sum(tot_rewards), Counter(tot_rewards))

100%|██████████| 100/100 [46:39<00:00, 27.99s/it]

231.5 Counter({3.5: 53, 1.0: 46, 0.0: 1})
